In [27]:
#Import statements

import os
import random
import gym
from gym import Env
import numpy as np
from gym.spaces import Discrete
from enum import Enum
from statistics import mean

ModuleNotFoundError: No module named 'gym'

In [38]:
#Battlefield setup

class Player():
    def __init__(self,name, decksize=60, num_creatures = 20, num_spells = 20) :
       self.name = name
       self.life = 20
       self.decksize = decksize
       self.battlefield = []
       self.deck = self.build_deck(num_creatures,num_spells)
       self.hand = []
       self.landforturn = False
       self.opponent_name='player 2' if self.name == 'player 1' else 'player 1'

    def build_deck(self):
        deck = []
        for i in range(num_creatures):
            deck.append('creature')
        for i in range(num_spells):
            deck.append('spell')
        for i in range(self.decksize - num_creatures - num_spells):
            deck.append('land')
        return deck
    
    def draw(self, number_draw = 1):
        self.landforturn = False
        for i in range(number_draw):
            self.hand.append(self.deck.pop(0))
            
    def attack(self, num_attackers):
        tapper = 0
        for k in range(len(self.battlefield)):
            if self.battlefield[k] == 0:
                if tapper < num_attackers:
                    self.battlefield[k] = 3
                    tapper += 1
                    
    def block(self, num_block, num_attack, opponent_player):
        counter = 0
        for k in range(num_block):
            if k+1 <= num_attack:
                opponent_player.battlefield.remove(3)
                self.battlefield.remove(0)
        if num_attack >= num_block:
            damage = (num_attack - num_block) * 2
            self.life -= damage
            print(f"{self.name} lost {damage} life.")
        else:
            print("overblocked")
            
    def play(self,  card_choice, opponent_player, target_choice = 0,):
        if card_choice == 'creature': 
            manapool = 0
            tapper = 0
            for k in range(len(self.battlefield)):
                if self.battlefield[k] == 'land':
                    if tapper < 2:
                        self.battlefield[k] = 'tapped_land'
                        tapper += 1
                self.hand.remove(card_choice)
                self.battlefield.append('creature')
        elif card_choice == 'spell': #spell
            # Play a spell, target choices are [p1, p2, p1 u_c, p1 t_c, p2 u_c, p2 t_C]
            # going from 1-6
            tapper = 0
            for k in range(len(self.battlefield)):
                if self.battlefield[k] == 'land':
                     if tapper < 1:
                        self.battlefield[k] = 'tapped_land'
                        tapper += 1
                # spell effects
                # Based on choice, remove that amount of life or that kind of creature from
                # appopriate battlefield

            match target_choice:
                case 1:
                    self.life = self.life - 3
                    print("Player 1's life total was targeted by a spell.")
                case 'b':
                    return 2
            if target_choice == 1:
                self.life = self.life - 3
                print("Player 1's life total was targeted by a spell.")
            elif target_choice == 2:
                self.opponent_player.life = self.opponent_player.life - 3
                print("Player 2's life total was targeted by a spell.")
            elif target_choice == 3:
                if 0 in self.battlefield: self.battlefield.remove(0)
                print("Player 1's untapped creature was targeted by a spell.")
            elif target_choice == 4:
                if 3 in self.battlefield: self.battlefield.remove(3)
                print("Player 1's tapped creature was targeted by a spell.")
            elif target_choice == 5:
                if 0 in self.opponent_player.battlefield: self.opponent_player.battlefield.remove(0)
                print("Player 2's untapped creature was targeted by a spell.")
            else:
                if 3 in self.opponent_player.battlefield: self.opponent_player.battlefield.remove(3)
                print("Player 2's tapped creature was targeted by a spell.")
                self.hand.remove(card_choice)
        elif card_choice == 'land':
            #if self.player1.landforturn == True:
                #print("You have already played a land for turn")
                #reward(penalty)
                #choose_card_choice([1])
            self.hand.remove(card_choice) #this needs to kick error, no land to play
            self.battlefield.append(card_choice)
            self.landforturn = True


class magicgame():
    def __init__(self,player1, player2):
        self.player1 = player1
        self.player2 = player2
        random.shuffle(self.player1.deck)
        random.shuffle(self.player2.deck)
        self.player1.draw(7)
        self.player2.draw(7)
        #phases for each player will be main phase 1, attacker phase, and blocking phase (1, 2, 3)
        self.phase = 1
        
    #Game Set up

    
    def show_battlefield(self):
        print("\n __________________________________________________________")
        print("\n", self.player2.hand)
        print("\n _____ P2 _____ Life: ", self.player2.life)
        for k in range(len(self.player2.battlefield)):
            #L = untapped land
            if self.player2.battlefield[k] == "land":
                print("L", end = " ")
            #l = tapped land
            elif self.player2.battlefield[k] == "tapped_land":
                print("l", end = " ")
        print("\n")
        for k in range(len(self.player2.battlefield)):
            #C = untapped creature
            if self.player2.battlefield[k] == "creature":
                print("C", end = " ")
            #c = tapped creature
            elif self.player2.battlefield[k] == "tapped_creature":
                print("c", end = " ")
        print("\n")
        for k in range(len(self.player1.battlefield)):
            if self.player1.battlefield[k] == "creature":
                print("C", end = " ")
            elif self.player1.battlefield[k] == "tapped_creature":
                print("c", end = " ")
        print("\n")
        for k in range(len(self.player1.battlefield)):
            if self.player1.battlefield[k] == "land":
                print("L", end = " ")
            elif self.player1.battlefield[k] == "tapped_land":
                print("l", end = " ")
        print("\n _____ P1 _____ Life: ", self.player1.life)
        print("\n", self.player1.hand)
        
    def p1_play(self, card_choice, target_choice = 0):
        if card_choice == 'creature': #creature
            manapool = 0
            #elif card_choice in self.player1.hand: #check if the card is in the players hand
            tapper = 0
            for k in range(len(self.player1.battlefield)):
                if self.player1.battlefield[k] == 'land':
                    if tapper < 2:
                        self.player1.battlefield[k] = 'tapped_land'
                        tapper += 1
                self.player1.hand.remove(card_choice)
                self.player1.battlefield.append('creature')
            #else:
                #print("You can't play that.")

        elif card_choice == 'spell': #spell
            # Play a spell, target choices are [p1, p2, p1 u_c, p1 t_c, p2 u_c, p2 t_C]
            # going from 1-6
            tapper = 0
            for k in range(len(self.player1.battlefield)):
                if self.player1.battlefield[k] == 'land':
                     if tapper < 1:
                        self.player1.battlefield[k] = 'tapped_land'
                        tapper += 1
                # spell effects
                # Based on choice, remove that amount of life or that kind of creature from
                # appopriate battlefield

            if target_choice == 1:
                self.player1.life = self.player1.life - 3
                print("Player 1's life total was targeted by a spell.")
            elif target_choice == 2:
                self.player2.life = self.player2.life - 3
                print("Player 2's life total was targeted by a spell.")
            elif target_choice == 3:
                if 0 in self.player1.battlefield: self.player1.battlefield.remove(0)
                print("Player 1's untapped creature was targeted by a spell.")
            elif target_choice == 4:
                if 3 in self.player1.battlefield: self.player1.battlefield.remove(3)
                print("Player 1's tapped creature was targeted by a spell.")
            elif target_choice == 5:
                if 0 in self.player2.battlefield: self.player2.battlefield.remove(0)
                print("Player 2's untapped creature was targeted by a spell.")
            else:
                if 3 in self.player2.battlefield: self.player2.battlefield.remove(3)
                print("Player 2's tapped creature was targeted by a spell.")
                self.player1.hand.remove(card_choice)


        elif card_choice == 'land':
            
            #if self.player1.landforturn == True:
                #print("You have already played a land for turn")
                #reward(penalty)
                #choose_card_choice([1])
            self.player1.hand.remove(card_choice) #this needs to kick error, no land to play
            self.player1.battlefield.append(card_choice)
            self.player1.landforturn = True
            #else:
                #print("You can't play that.")
    
    def nextphase(self):
        self.phase += 1
        if self.phase == 4:
            self.phase = 11
        elif self.phase == 14:
            self.phase = 1
        else:
            pass


        
    #Start of turn

    def p1_turn(self):
        for k in range(len(self.player1.battlefield)):
            if self.player1.battlefield[k] == 3:
                    self.player1.battlefield[k] = 0
            if self.player1.battlefield[k] == 4:
                    self.player1.battlefield[k] = 2
        p1_draw()

    def p2_turn(self):
        for k in range(len(self.player2.battlefield)):
            if self.player2.battlefield[k] == 3:
                    self.player2.battlefield[k] = 0
            if self.player2.battlefield[k] == 4:
                    self.player2.battlefield[k] = 2
        p2_draw()

    

In [ ]:
#Player Actions

def chooseAction(value, prob):
        decision = np.random.choice(value, p = prob)
        return decision

In [26]:

def playgame():
    game = magicgame()
    while (game.player1.life > 0 and game.player2.life > 0) and (len(game.player2.deck) > 0):
        # p1 phase 1
        #call creature tracker, set output = previous_state_tup
        if game.phase == 1:
            # show the battle field
            game.show_battlefield()
            print("What does P1 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     2 = land, 8 = pass phase")
            
            numConv = {"creature":0, 
                        "spell":1, 
                         "land":2, 
                          "P1 T creature":4,
                          "P2 U creature":5, 
                         "P2 T creature": 6,
                          "pass phase":8}
            
            p1_decision = None
            if game.player1.battlefield.count("land") >= 2:
                if game.player1.landforturn == False:
                    if "land" in game.player1.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("creature" in game.player1.hand) and ("spell" not in game.player1.hand):
                            values.append("creature")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player1.hand) and ("creature" not in game.player1.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player1.hand) and "creature" in game.player1.hand:
                            values.append("spell")
                            values.append("creature")
                            prob = [0.7,0.1,0.1,0.1]
                        p1_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.p1_hand))
                        num_choices.append("pass phase")
                        p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.p1_hand).difference(set([2])))
                    num_choices.append("pass phase")
                    p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            elif game.player1.battlefield.count("land") == 1:
                if game.player1.landforturn == False:
                    if "land" in game.p1_hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("spell" in game.player1.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        else:
                            print("You only have one untapped mana. You can't play a creature and you don't have a spell.")
                        p1_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.player1.hand).difference(set(["creature"])))
                        num_choices.append("pass phase")
                        p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.player1.hand).difference(set(["creature","land"])))
                    num_choices.append("pass phase")
                    p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            else:
                if game.player1.landforturn == False:
                    if "land" in game.p1_hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        p1_decision = chooseAction(values,prob)
                    else:
                        p1_decision = "pass phase"
                else:
                    p1_decision = "pass phase"
                    print("No more moves. Pass the turn")
            
            p2_decision = None
            
        # Stopped here

            # If player one plays a spell
            if p1_decision == "spell":
                # Player one choose target
                print("Target?")
                #######
                ## These numbers needs to be discussed
                values = [1,2]
                probs = [.05,.95]

                p1_set_creatures = set(game.player1.battlefield).difference(set([4,2]))
                p2_set_creatures = set(game.player2.battlefield).difference(set([4,2]))
                if len(p1_set_creatures) != 0 and len(p2_set_creatures) == 0:
                    probs = [.05, .85]
                    temp_probs = [.1/len(p1_set_creatures)] * len(p1_set_creatures)
                    probs += temp_probs
                    if 0 in game.p1_battlefield:
                        values.append(3)
                    if 3 in game.p1_battlefield:
                        values.append(4)
                elif len(p1_set_creatures) == 0 and len(p2_set_creatures) != 0:
                    probs = [.05]
                    temp_probs = [.95/(len(p2_set_creatures)+1)] * (len(p2_set_creatures)+1)
                    probs += temp_probs
                    if 0 in game.p2_battlefield:
                        values.append(5)
                    if 3 in game.p2_battlefield:
                        values.append(6)
                elif len(p1_set_creatures) != 0 and len(p2_set_creatures) != 0:
                    probs = [.05]
                    temp = .85/(len(p2_set_creatures)+1)
                    probs.append(temp)
                    temp_probs_p1 = [.1/len(p1_set_creatures)]*len(p1_set_creatures)
                    probs += temp_probs_p1
                    temp_probs_p2 = [temp]*len(p2_set_creatures)
                    probs += temp_probs_p2
                    if 0 in game.p1_battlefield:
                        values.append(3)
                    if 3 in game.p1_battlefield:
                        values.append(4)
                    if 0 in game.p2_battlefield:
                        values.append(5)
                    if 3 in game.p2_battlefield:
                        values.append(6)
                #######
                target = np.random.choice(values, p = probs) #P1 wants to target P2 stuff
                game.p1_play(p1_decision, target)
            elif p1_decision == 8:
                #update needs to be here as well
                game.nextphase()
            else:
                game.p1_play(p1_decision)
                
            #update decision probs
            
        elif game.phase == 2:
            game.show_battlefield()
            print("How many P1 attackers? Any N attackers, 0 is pass")
            #global num_attack
            print(str(game.p1_battlefield.count(0)) + " p1 battlefield")
            values = list(range(0,game.p1_battlefield.count(0)+1))
            probs = [1/(game.p1_battlefield.count(0)+2)]*(game.p1_battlefield.count(0))
            probs.append(2/(game.p1_battlefield.count(0)+2))
            num_attack = np.random.choice(values, p = probs) # needs to be random
            print("Player 1 is attacking with " + str(num_attack) + " creatures.")
            game.p1_attack(num_attack)
        elif game.phase == 3:
            if num_attack == 0:
                game.nextphase()
                pass
            else:
                game.show_battlefield()
                print("How many P2 blockers? Any N blockers, 0 is pass")
                values = list(range(0,game.p2_battlefield.count(0)+1))
                probs = [1/(game.p2_battlefield.count(0)+2)]*(game.p2_battlefield.count(0))
                probs.append(2/(game.p2_battlefield.count(0)+2))
                p2_decision = np.random.choice(values,p = probs) # needs to be random
                print("Player 2 is blocking with " + str(p2_decision) + " creatures.")
                game.p2_block(p2_decision, num_attack)
            game.p2_turn()
            game.p2_draw()

        if game.phase == 11:
            game.show_battlefield()

            print("What does P2 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     2 = land, 8 = pass phase")

            p1_decision = None
            ####
            if game.p2_battlefield.count(2) >= 2:
                if game.p2_landforturn == False:
                    if 2 in game.p2_hand:
                        values  = [2,8]
                        prob = [0.95,0.05]
                        if (0 in game.p2_hand) and (1 not in game.p2_hand):
                            values.append(0)
                            prob = [0.7,0.05,0.25]
                        elif (1 in game.p2_hand) and (0 not in game.p2_hand):
                            values.append(1)
                            prob = [0.7,0.05,0.25]
                        elif (1 in game.p2_hand) and 0 in game.p2_hand:
                            values.append(1)
                            values.append(0)
                            prob = [0.7,0.1,0.1,0.1]
                        p2_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.p2_hand))
                        num_choices.append(8)
                        p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.p2_hand).difference(set([2])))
                    num_choices.append(8)
                    p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            elif game.p2_battlefield.count(2) == 1:
                if game.p2_landforturn == False:
                    if 2 in game.p2_hand:
                        values  = [2,8]
                        prob = [0.95,0.05]
                        if (1 in game.p2_hand):
                            values.append(1)
                            prob = [0.7,0.05,0.25]
                        else:
                            print("You only have one untapped mana. You can't play a creature and you don't have a spell.")
                        p2_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.p2_hand).difference(set([0])))
                        num_choices.append(8)
                        p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.p2_hand).difference(set([0,2])))
                    num_choices.append(8)
                    p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            else:
                if game.p2_landforturn == False:
                    if 2 in game.p2_hand:
                        values  = [2,8]
                        prob = [0.95,0.05]
                        p2_decision = chooseAction(values,prob)
                    else:
                        p2_decision = 8
                else:
                    p2_decision = 8
                    print("No more moves. Pass the turn")
            ####

            if p2_decision == 1:
                print("Target?")
                ###
                values = [1,2]
                probs = [.95,.05]

                p1_set_creatures = set(game.p1_battlefield).difference(set([4,2]))
                p2_set_creatures = set(game.p2_battlefield).difference(set([4,2]))
                if len(p1_set_creatures) != 0 and len(p2_set_creatures) == 0:
                    probs = []
                    temp = .95/(len(p1_set_creatures)+1)
                    probs.append(temp)
                    probs.append(.05)
                    temp_probs = [temp] * (len(p1_set_creatures))
                    probs += temp_probs
                    if 0 in game.p1_battlefield:
                        values.append(3)
                    if 3 in game.p1_battlefield:
                        values.append(4)
                elif len(p1_set_creatures) == 0 and len(p2_set_creatures) != 0:
                    probs = [.85,.05]
                    temp_probs = [.1/len(p2_set_creatures)] * len(p2_set_creatures)
                    probs += temp_probs
                    if 0 in game.p2_battlefield:
                        values.append(5)
                    if 3 in game.p2_battlefield:
                        values.append(6)
                elif len(p1_set_creatures) != 0 and len(p2_set_creatures) != 0:
                    probs = []
                    temp = .85/(len(p1_set_creatures)+1)
                    probs.append(temp)
                    probs.append(.05)
                    temp_probs_p1 = [temp]*len(p1_set_creatures)
                    probs += temp_probs_p1
                    temp_probs_p2 = [.1/len(p2_set_creatures)]*len(p2_set_creatures)
                    probs += temp_probs_p2
                    if 0 in game.p1_battlefield:
                        values.append(3)
                    if 3 in game.p1_battlefield:
                        values.append(4)
                    if 0 in game.p2_battlefield:
                        values.append(5)
                    if 3 in game.p2_battlefield:
                        values.append(6)
                ###
                target = np.random.choice(values, p = probs) #P2 wants to target P1 stuff
                game.p2_play(p2_decision, target)
            elif p2_decision == 8:
                game.nextphase()
            else:
                game.p2_play(p2_decision)
        elif game.phase == 12:
            game.show_battlefield()
            print("How many P2 attackers? Any N attackers, 0 is pass")
            print(str(game.p2_battlefield.count(0)) + " p2 battlefield")
            num_attack = random.randint(0,game.p2_battlefield.count(0)) # needs to be random
            print("Player 2 is attacking with " + str(num_attack) + " creatures.")
            print(num_attack)
            game.p2_attack(num_attack)
        elif game.phase == 13:
            if num_attack == 0:
                game.nextphase()
                pass
            else:
                game.show_battlefield()
                print("How many P1 blockers? Any N blockers, 0 is pass" )
                p1_decision = random.randint(0,game.p1_battlefield.count(0)) # needs to be random
                print("Player 1 is blocking with " + str(p1_decision) + " creatures.")
                game.player1.block(p1_decision, num_attack)
            game.p1_turn()
            game.player1.draw()
        else:
            pass
    winner = show_winner(game.p1_life, game.p2_life, game.p2_deck)
    return winner

TypeError: __init__() missing 2 required positional arguments: 'player1' and 'player2'

In [41]:
match point:
    case (0, 0):
        print("Origin")
    case (0, y):
        print(f"Y={y}")
    case (x, 0):
        print(f"X={x}")
    case (x, y):
        print(f"X={x}, Y={y}")

SyntaxError: invalid syntax (<ipython-input-41-8d83f35df01c>, line 1)